## Coinbase Pro (GDAX) Account Report.
Contains information about all orders (buys,sells,trades) made using, and transfers into an out of, the Coinbase Pro (formerly GDAX) platform.
### How to get:
- login to Coinbase.com
- Go to: https://accounts.coinbase.com/profile 
- Select "statements"
- Click on the "Coinbase Pro" tab
- "Generate custom report" with:
    - "account"
    - "all portfolios"
    - "all accounts"
    - Select the desired year



An "order" is any trade, purchase, or sale that has occurred. A single order ("sell 3 BTC for USD") may have been performed by executing multiple smaller "trades" - but it's the order we are interested.

This code will parse the report and assemble a list of `OrderInfo` instances describing the orders.

All of the records for a given order have the same "order id" field entry.

There will be at least 1, and may be many, "trades" in an order. All records for a given trade will have the same "trade id" value.

Every trade cosists of 2 "match" records, one with a positive "amount" which represents the item received, and another with a negative "amount", representing the item or currency paid of given in exchange.

A trade may or may not have a "fee" record - describing any fee paid.

For each order we want:

- market:        'BTC-USD', for instance
- timestamp:     the timestamp of the latest executed trade (arbitrary choice, in reality)
- order_id:      the coinbase order ID
- unit_given:     what was sold/given? ('BTC', 'ETH'...)
- amount_given:   the amount of the currency being sold
- unit_received: payment unit ( 'USD', USDC, ETH... )
- amount_received: the amount received in the trades (no fees)
- unit_price:    In practice this is calculated as (amount_received / amount_given) 
- fees_paid:     the total fees for the order


In [1]:
# allow import of local fifo-tool stuff
import os
import sys
sys.path.insert(0, os.path.abspath('../src'))

In [2]:
from typing import Dict,List
from datetime import datetime
import json
import numpy as np
import pandas as pd

from models.acquisition import Acquisition
from models.disposition import Disposition
from models.stash import Stash

In [3]:
def read_report_csv(file_path):
    """Read a coinbase pro 'accounts' report and return a pandas dataframe.
        conversions done:
            'time' - parsed into a datestamp
            'trade id' - read as a string
    """
    date_flds = ['time']
    forced_dtypes = {'trade id': str}
    return pd.read_csv(file_path, skiprows=0, parse_dates=date_flds, dtype=forced_dtypes)


In [4]:
#help(pd.read_csv)

In [5]:
def get_orders(accounts_rpt):
    """Given a report dataframe, select the appropriate records and group them
        into per-order dataframes. Return a list of them
    """
    mask = (accounts_rpt['type']=='match') | (accounts_rpt['type']=='fee')
    matches = accounts_rpt[mask] # get the matches and fees
    orders_df = matches.groupby('order id') # group into orders
    orders = [orders_df.get_group(id) for id in  orders_df.groups.keys()]
    return orders


In [10]:

class CbProOrderInfo:

    def __init__(self, order_id: str, timestamp: float, unit_given: str,
                unit_received: str, amount_given: float, amount_received: float,
                unit_price: float,  fees: float) -> "OrderInfo":
        self.order_id: str = order_id
        self.timestamp: float = timestamp
        self.unit_given: str = unit_given
        self.unit_received: str = unit_received
        self.amount_given = amount_given
        self.amount_received = amount_received
        self.unit_price = unit_price
        self.fees = fees

    def to_json_dict(self) -> Dict:
        return  {
            "order_id": self.order_id,
            "timestamp": self.timestamp,
            "unit_given": self.unit_given,
            "unit_received": self.unit_received,
            "amount_given": self.amount_given,
            "amount_received": self.amount_received,
            "unit_price": self.unit_price,
            "fees": self.fees
        }

    def to_disposition(self, asset) -> 'Disposition':
        assert self.unit_given == asset, f'Order is not a {asset} disposition'
        return Disposition(
            self.timestamp,
            self.unit_given, # asset_type sold
            self.amount_given, # asset_amount
            self.unit_price, # asset_price,
            self.fees, # fees
            f'CB Pro Order Id: {self.order_id}', # reference
            "" #comment
        )

    def to_acquisition(self, asset) -> 'Acquisition':
        assert self.unit_received == asset, f'Order is not a {asset} acquisition'
        return Acquisition(
            self.timestamp,
            self.unit_received, # asset_type bought
            self.amount_received, # asset_amount
            self.unit_price, # asset_price,
            self.fees, # fees
            f'CB Pro Order Id: {self.order_id}', # reference            
            "" #comment
        )

def parse_order(order, asset):
    """ Parse the trades in an order dataframe.

        returns a CbProOrderInfo instance if the order involves the given asset
        An order is a DataFrame

    """
    ID_LBL = 'order id'
    UNIT_LBL = 'amount/balance unit'
    order_id = order[ID_LBL].values[0]  # is in every record
    timestamp = max(order['time']).timestamp()
    # matches are about the item, fees are about fees
    matches = order[order['type'] == 'match']
    fees =  order[order['type'] == 'fee']

    # The item being received (bought, usually) has a positive amount, 
    # the one given has a negative one
    units = np.unique(order[UNIT_LBL].values) 
    # units 2-element array containing the 2 units
    mask0 = order[UNIT_LBL] == units[0]
    mask1 = order[UNIT_LBL] == units[1]
    amounts = (order[mask0]['amount'].values.sum(), order[mask1]['amount'].values.sum())
    # we are going to assume that 1 of the amounts is negative, the oher positive
    (given_idx, rcvd_idx) = (0,1) if amounts[1]>0 else (1,0)
    unit_given = units[given_idx]
    amount_given = abs(amounts[given_idx])
    unit_received = units[rcvd_idx]
    amount_received = abs(amounts[rcvd_idx])
    # "price" is always cash/asset, so we do need to know whether received or given is the asset
    if unit_given == asset:    
        unit_price = amount_received/amount_given # we sold crypto
    else:
        unit_price = amount_given/amount_received # we bought crypto
    fees = abs(fees['amount'].sum()) # fees are reported as < 0
    if unit_given == asset or unit_received == asset:  # ignore if it's  not the asset we're tracking 
        return CbProOrderInfo(order_id, timestamp, unit_given, unit_received,
                              amount_given, amount_received, unit_price, fees)
    else:
        return None



In [11]:
def process_file( year: str, assets: List[str]) -> None:
    filebase = f'local_data/cbpro-account-{year}'
    main_df = read_report_csv(filebase+'.csv')
    orders = get_orders(main_df)
    
    for asset in assets:
        infos = [i for i in [parse_order(o, asset) for o in orders] if i] # filter out Nones
        acqs = []
        disps = []
        for info in infos:
            if info.unit_given == asset:  # it's a sale/disposition
                disps.append( info.to_disposition(asset) )
                #print("Disp!")
            else: # purchase/acquisition
                acqs.append( info.to_acquisition(asset) )
                #print("Acq!")
        data = Stash(asset, f"Coinbase Pro {asset} orders - {year}", acqs, disps)
        
        #json.dumps(jd)
        with open(filebase+f'-{asset}.json', 'w') as f:
            jd = data.to_json_dict()
            json.dump(jd, f, indent=2)

In [12]:
for yr in ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']:
    process_file(yr, ['BTC', 'ETH'] ) 

In [28]:
# Working/testing/messing about starts here

,portfolio,type,time,amount,balance,amount/balance unit,transfer id,trade id,order id
0,default,deposit,2020-08-21 03:15:15.538000+00:00,100.000000,100.006403,USD,cae094cd-f0c6-4ee2-b5b2-e1b6de564b11,NaN,NaN
1,default,deposit,2020-08-21 15:24:53.903000+00:00,1000.000000,1100.006403,USD,e2a50729-4593-414a-a4b9-c807fd150a46,NaN,NaN
2,default,match,2020-08-24 14:41:23.275000+00:00,-17.786148,1082.220255,USD,NaN,62632152,7e0c4b6f-6f82-47fa-8f69-7ffceba19719
3,default,match,2020-08-24 14:41:23.275000+00:00,0.043971,0.043971,ETH,NaN,62632152,7e0c4b6f-6f82-47fa-8f69-7ffceba19719
4,default,fee,2020-08-24 14:41:23.275000+00:00,-0.088931,1082.131324,USD,NaN,62632152,7e0c4b6f-6f82-47fa-8f69-7ffceba19719
5,default,match,2020-08-24 14:41:24.810000+00:00,-117.809816,964.321508,USD,NaN,62632153,7e0c4b6f-6f82-47fa-8f69-7ffceba19719
6,default,match,2020-08-24 14:41:24.810000+00:00,0.291248,0.335219,ETH,NaN,62632153,7e0c4b6f-6f82-47fa-8f69-7ffceba19719
7,default,fee,2020-08-24 14:41:24.810000+00:00,-0.589049,963.732459,USD,NaN,62632153,7e0c4b6f-6f82-47fa-8f69-7ffceba19719
8,default,match,2020-08-24 14:41:27.937000+00:00,-4.655601,959.076858,USD,NaN,62632154,7e0c4b6f-6f82-47fa-8f69-7ffceba19719
9,default,match,2020-08-24 14:41:27.937000+00:00,0.011510,0.346728,ETH,NaN,62632154,7e0c4b6f-6f82-47fa-8f69-7ffceba19719


In [169]:
o = orders[2]
infos[0].to_json_dict()

{'order_id': '244e6454-dfab-4471-a5bb-a16ac416cdca',
 'timestamp': 1449116799.73,
 'unit_given': 'BTC',
 'unit_received': 'USD',
 'amount_given': -3.0,
 'amount_received': 1102.5,
 'unit_price': -0.0027210884353741495,
 'fees': 0.0}

In [121]:
#matches = o[o['type']=='match']
#matches

In [131]:
units = np.unique(o['amount/balance unit'].values)
units

array(['BTC', 'USD'], dtype=object)

In [398]:
o['order id'].values[0] # order id

'244e6454-dfab-4471-a5bb-a16ac416cdca'

In [136]:
mask0 = o['amount/balance unit'] == units[0]
mask1 = o['amount/balance unit'] == units[1]
o[mask0]['amount'].values.sum(),  o[mask1]['amount'].values.sum(),

(-5.0, 1969.6914845000001)

In [400]:
o['amount/balance unit'].values[1] # currency used

'USD'

In [401]:
max(o['time']) # timestamp

Timestamp('2015-12-03 04:26:39.730000+0000', tz='UTC')

In [402]:
btc_mask = o['amount/balance unit']=='BTC'

In [403]:
o[btc_mask]['amount'].sum() # amount of BTC

-3.0

In [404]:
o[~btc_mask]['amount'].sum() # amount of $

1102.5

In [405]:
[ o[o['amount/balance unit']!='BTC']['amount'].sum() for o in orders]

[1102.5,
 4400.0,
 1969.6914845000001,
 4300.0,
 3700.0,
 4257.5,
 4750.0,
 1950.0,
 9093.25]

In [382]:
mask1 = main_df['type']=='match'

In [383]:
mask2 = main_df['type']=='fee'

In [384]:
mask1.value_counts()

type
True     100
False     35
Name: count, dtype: int64

In [385]:
mask2.value_counts()

type
False    117
True      18
Name: count, dtype: int64

In [390]:
foo = mask1 | mask2
foo.values, foo.value_counts()

(array([False,  True,  True, False, False,  True,  True, False, False,
         True,  True,  True,  True, False,  True,  True, False, False,
        False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False, False,
         True,  True, False, False,  True,  True,  True,  True,  True,
      

In [387]:
mask1.values

array([False,  True,  True, False, False,  True,  True, False, False,
        True,  True,  True,  True, False,  True,  True, False, False,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True, False,

In [379]:
mask2.values

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [391]:
foo.values

array([False,  True,  True, False, False,  True,  True, False, False,
        True,  True,  True,  True, False,  True,  True, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True, False,